# **IEOR 4650  Business Analytics (Fall 2019)**

**Homework 2 [Total points: 60]**

** Due: 11:59 PM, October 8**

This second homework consists of Python programming practices and the applications to linear regression. 

The steps to finish in the homework:

*   Step 1: Make a copy of the file to your LionDrive.
*   Step 2: Work with the copy (an ipynb file).
*    <font color='red'>**Step 3: Rename the copy to your_uni_hw2.ipynb (For example: yd4501_hw2.ipynb).**</font>  
*   Step 4: Submit the copy on CourseWorks.

Before you submit your assignment, make sure to re-run your code from the beginning. (You can do so by first clicking Runtime/Reset All Runtimes and then clicking Runtime/Run all)


**All the questions in this homework requires coding**

In [0]:
#This imports all the packages you need for the homework
#Please run this first


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#import modules

import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt




##Setting
In this exercise, let's continue working with a dataset which explores the store sales.

Before doing the assignment, please study this dataset here https://www.kaggle.com/c/rossmann-store-sales/data


For this study, we are interested in finding a good model that gives good prediction performance. 

* Especially, we are interested in use RMSPE as our accuracy measurement. RMSPE is defined as following:
$$RMSPE = \sqrt{\frac{\frac{(y_i-\widehat{y_i}}{y_i})^2}{N}}$$
* In addition, any day and store with 0 actual sales is ignored in scoring.

In [0]:
#Load the dataset
link="https://drive.google.com/open?id=1Q6J0q4tlWJ7TajX6hMSvZ8tK3CyDD8QY"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Sales.csv')  
SALE = pd.read_csv('Sales.csv')
SALE.head(5)



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


*You might see a warning when importing the data. This is because StateHoliday has both values in string and numbers. No worries this for now.*

In [0]:
link="https://drive.google.com/open?id=1g6URzUJnhCLOtg0a9TTHC18KYcqwxvoc"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('myfile_store.csv')  
STORE = pd.read_csv('myfile_store.csv')
STORE.head(5)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Q1. [8 points]
**[Data Cleaning]**

1. Merge two tables based on "Store" column

2. To save you from the pain of running the models for too long, **keep only records for store 1-100**.

3. This study only cares about the accuracy performend on the data with Sales higher than 0.There are pros and cons of keeping 0 Sales records. For this study, let's **delete all the sales records with sales=0**.

4. Print out the unique values you have in "StateHoliday" column. If you see anything that goes wrong, fix the issue. After that, print out the unique values again in "StateHoliday" column. [You might find this [link](https://www.geeksforgeeks.org/python-pandas-series-str-replace-to-replace-text-in-a-series/) useful]


## Q2 [4 points]

**[Preparation for cross-validation]**

Split our data into three segments.Instead of shuffling them first, we will do something different. When working with prediction, we might want to have a model that can give good prediction power for the future sales. Thus, we will split our data in the following way:

(1) Training: Time window 2013-01-01 to 2015-03-31  

(2) Validation: Time window 2015-04-01 to 2015-05-31 

(3) Testing: Time window 2015-06-01 to 2015-07-31

Print out how many records you have for each set.

In [0]:
#You might find the following sample code helpful 
#X=X.astype({"Date":"datetime64"})
#X["Date"]<"2015-01-02"

##Q3 [8 points]

**[Feature Engineering 1 ]**

Usually, people pay great attention to the features that are highly predictive. Making sure that adequate variable transformations are performed on those variables are highly important. 

* Examine why customer numbers is an important predictor based on the data.

* Use 4 histograms show how taking the log-transformation changes the distribution of "Sales" and "Customers" on the training set. Make layout the 4 histograms following the following format

\begin{bmatrix}
    \text{Sales plot here} &  \text{log(Sales) plot here} \\
    \text{Customer plot here} &  \text{log(Customer) plot here}
\end{bmatrix}.

>You will find this [link](https://matplotlib.org/3.1.0/gallery/subplots_axes_and_figures/subplots_demo.html) useful

2. 
  Based on cross-validation to show which model gives a better performance. 
  $$log(Sales)=\beta_0+\beta_1 log(Customers)+\epsilon$$ 
  $$Sales=\beta_0+\beta_1 Customers+\epsilon$$ 


##Q4 [7 points]

**[Feature Engineering 2]**

In the business operation, stores performance can vary with time. Including the time/date information can potentially help us increase the model fit. Some typical factors we can consider are:

* Year [store performance in 1992 can be different from the performance in 2010]
* Month [For example, an ice-cream shop generally have higher sales in the summer months]
* Day of the week [Especially, weekend v.s. weekdays might show very differen performances]
* Important events

In our dataset, we have "DayOfWeek", "SchoolHoliday" and "StateHoliday" to capture the last two factors. However, we do not have the "Year" and "Month".

Luckily, once we specify "Date" as the datetime type. We can easily extract the time information. See the link [here](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetimelike-properties)

For Q4: Do the following 

* Argue the pros and cons of adding Month as a categorical variable v.s. continuous variable.
* Determine the best decision using cross-validation.


In [0]:
#Suppose x3 is a datetype column. If you want to get year information and treat it as a continuous variable in the regression, you can simply do
#("y~x1+x2+x3.dt.month",data=Data)


##Q5 [10 points]
### [Forward Model Selection]

Time to practice some model selection technique. Let's suppose that we want to determine some predictors we would like to adding based on the baseline model you chose in **Q3**. [To reduce your coding complexity, let's simply consider the following three factors]

* Promo	
* Promo2
* SchoolHoliday


Based on cross-validation, choose the best model. 
* Report RMSPE of each model and the new variable to be selected in each round of model selection
[You can do this manually]

* After selecting the final model, report the accuracy on the test set.

##Q6 [10 points]
###[Ridge regression]

When working with dataset (especially if we are using many predictors for a small dataset), we might encounter some serious problem with over-fitting.

To demonstrate how to use Ridge regression to deal with this problem. We will explore a setting when we have very limited data. In this dataset, let's simply use the information from **Store 3** from 2013-01-02 to 2013-02-18. 

Again, we will split our data into three segments based on the date.

* Training: Information for Store 3 before 2013-02-02
* Validation: Information for Store 3 between 2013-02-2 and 2013-02-12-11
* Testing: Information from Store 3 between 2013-02-12 and 2013-02-18


Conduct ridge regression on the training set. 

Assume the model you want to run is 
$$log(Sale)=\beta_0+\beta_1 log (Customers) + \beta_2 Customers +\epsilon$$


* Plot the relationship between $\alpha$ and RMSPE on the Validation set. 


##Q7. [13 points]
###[Standardization in ridge regression]
The previous model gives an okay solution. However, one thing to notice is that the scale of variations could also affect $\beta$ values, which can cause factors to be discriminated simply based on whether the values are small or large. In addition, we do not want $\beta_0$ to be penalized.

As a result, people usually standardize the variables. Especially, for Ridge regression, a common practice is to do the following:

**On the training set:**

1. Standardize each independent variable using 

$$\frac{x_k-\bar{x_k}}{s_{x_k}}$$

where $\bar{x}$ and $s_x$ are the mean and the standard deviation of variable $x_k$ on the training set. 


2. Re-center the dependent variable using 

$$y-\bar{y}$$

where $\bar{y}$ is the mean of $y$ on the training set.

This allows us to tease out the impact of $\beta_0$ in the regulation

3. Regress the de-centered depedent variable on the standardized independent variables without intercept. [you can simply use "y~x1+x2-1" to not estimate the intercept]

**On the validation and testing set:**

When you do the prediction on the validation set, we will have

$$\hat{y}=\bar{y}+\sum \hat{\beta_k} {\frac{x-\bar{x_k}}{s_{x_k}}}$$

where $\bar{y}$ is the mean of $y$ on the **training set**, $\bar{x_k}$ is the mean of $x_k$ on the **training set**, while $s_{x_k}$ is the standard deviation of $x_k$ on the **training set**.



* Use this method to re-do Q6. 
* Determine the best model based on Q6 and Q7. Report the accuracy on the test set.



##Q*. Bonus question [3 points]

Do the task in Q5 but with the following factors with the help of `for` loop

* Promo
* SchoolHoliday
* C(Store)
* C(DayOfWeek)
* C(Store):np.log(Customers)
* C(Month)
* C(Year)
* C(StateHoliday)

